# Imports

In [1]:
import sklearn.neural_network._multilayer_perceptron as mlp
import pyarrow as pa
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets as datasets
import sklearn.model_selection as model_selection
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preprocessing
import sklearn.metrics as metrics
from sklearn.metrics import r2_score
import pandas as pd
from numpy import random as nprandom 
import random
import scipy as sp
import warnings

# From example
from warnings import catch_warnings, simplefilter

In [2]:
# suppress specific warnings
warnings.filterwarnings("ignore", message="ABNORMAL_TERMINATION_IN_LNSRCH.")
warnings.filterwarnings("ignore", message="Creating children from parents .* in generation .* in loop .*")

# Data preparation

## Import the dataset
Import the dataset into a pandas dataframe

In [3]:
dataset = pd.read_csv('UNSW_NB15.csv')
dataset.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,...,1,2,0,0,0,1,2,0,Normal,0
1,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,...,1,2,0,0,0,1,2,0,Normal,0
2,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,...,1,3,0,0,0,1,3,0,Normal,0
3,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,...,1,3,0,0,0,2,3,0,Normal,0
4,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,...,1,3,0,0,0,2,3,0,Normal,0


In [4]:
# take a sample of the data
dataset = dataset.sample(frac=0.1)


# clean the data
dataset = dataset.drop_duplicates()
dataset = dataset.dropna()
dataset = dataset.drop(columns=['attack_cat'])
dataset = pd.get_dummies(dataset, columns=['proto', 'service', 'state'])
dataset.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_CON,state_FIN,state_INT,state_REQ,state_RST
23237,0.000003,2,0,114,0,333333.321500,254,0,1.520000e+08,0.000000,...,False,False,False,False,False,False,False,True,False,False
42923,0.926973,16,12,904,642,29.127062,254,252,7.318444e+03,5083.211914,...,False,False,False,False,False,False,True,False,False,False
201424,0.000008,2,0,114,0,125000.000300,254,0,5.700000e+07,0.000000,...,False,False,False,False,False,False,False,True,False,False
69801,0.369030,6,2,1012,86,18.968648,62,252,1.829662e+04,932.173523,...,False,False,False,False,False,True,False,False,False,False
41825,54.628231,20,16,1176,708,0.640694,254,252,1.637249e+02,97.239098,...,False,False,False,False,False,False,True,False,False,False


## Data omzetten in X en y

In [5]:
X = dataset.drop(columns=['label'])
y = dataset['label']

# Model generator

In [6]:
class MLPModelGenerator:
    def __init__(self, genetic_pool=None, fitness=True):
        self.genetic_pool = genetic_pool
        if genetic_pool is None:
            self.genetic_pool = {
                'hidden_layer_sizes': [int(i) for i in sp.stats.norm.rvs(loc=20, scale=5, size=nprandom.randint(2, 5))], # uses normal distribution to generate hidden layers.
                'activation': nprandom.choice(['identity', 'logistic', 'tanh', 'relu']),
                'solver': nprandom.choice(['lbfgs', 'sgd', 'adam']),
                'alpha': nprandom.uniform(low=0.0001, high=0.1),
                'batch_size': nprandom.randint(200, 800),
                'learning_rate': nprandom.choice(['constant', 'invscaling', 'adaptive']),
                'learning_rate_init': nprandom.uniform(low=0.001, high=0.1),
                'power_t': nprandom.uniform(low=0.1, high=0.9),
                'max_iter': nprandom.randint(200, 500),
                'shuffle': bool(nprandom.randint(0, 1)),
                'random_state': nprandom.randint(0, 100),
                #'tol': nprandom.uniform(low=1e-5, high=1e-3), # 1e-5 to 1e-3 is the range of tol values, one of the hyperparameters of the Adam optimizer
                #'verbose': bool(nprandom.randint(0, 2)),
                #'warm_start': bool(nprandom.randint(0, 2)),
                'momentum': nprandom.uniform(low=0.1, high=0.9),
                'nesterovs_momentum': bool(nprandom.randint(0, 2)),
                #'early_stopping': bool(nprandom.randint(0, 2)),
                #'validation_fraction': uniform(low=0.1, high=0.3),
                'beta_1': nprandom.uniform(low=0.1, high=0.999),
                'beta_2': nprandom.uniform(low=0.1, high=0.999),
                'epsilon': nprandom.uniform(low=1e-8, high=1e-7), # 1e-8 to 1e-7 is the range of epsilon values, one of the hyperparameters of the Adam optimizer
                'n_iter_no_change': nprandom.randint(10, 20),
                #'max_fun': nprandom.randint(15000, 20000),
            }
        if fitness:
            self.fitness_score = self.generate_fitness_score()
        else:
            self.fitness_score = 0

    def generate_fitness_score(self):
        try:
            model = mlp.MLPClassifier(**self.genetic_pool)
            model.fit(X, y)
            return model.score(X, y)
        except Exception as e:
            print(f"An error occurred: {e}")
            return 0


# Genereer waardes

## Genepool

In [7]:
initial_genepool_size = 10
initial_genepool = [MLPModelGenerator(fitness=False) for _ in range(1, initial_genepool_size)] # generate initial genepool based on the initial_genepool_size

## Overige waardes

In [8]:
desired_fitness_score = 1
current_generation = 0
max_generations = 10

This function is a genetic algorithm designed to optimize the hyperparameters of a MLP (Multi-Layer Perceptron) model.
The genetic algorithm is implemented using a list of MLPModelGenerator objects, which represent different models with different hyperparameters.
The genetic algorithm aims to evolve the models in the list to find the best set of hyperparameters that maximize the fitness score of the model.
It does this by using a while loop to iterate through the generations until either the maximum number of generations is reached or a model with a fitness score equal to or greater than the desired fitness score is found.
Within each iteration of the loop, the current generation is sorted based on the fitness score of the models in descending order. This ensures that the models with higher fitness scores are at the beginning of the list.

If the model with the highest fitness score in the current generation meets or exceeds the desired fitness score, the loop is terminated.
Otherwise, a new generation is created by performing genetic operations such as crossover and mutation.
The crossover operation involves selecting two parents from the current generation and creating two children by combining their genetic information. The genetic information is selected randomly from either parent based on a random binary choice.

The new generation, represented by the new_genepool list, is populated with the created children. The process continues until the new_genepool has the same size as the current_generation.
After creating the new generation, a mutation operation is applied to each model in the new_genepool with a probability of 0.1. This mutation operation introduces random changes to the model's genetic information.

Finally, the new_genepool is returned as the output of the function.
    

In [9]:
def genetic_algorithm(current_generation, max_generations, current_generation_number, desired_fitness_score) -> list:
    """
    Genetic algorithm to optimize hyperparameters of a MLP model
    
    
    Parameters:
    current_generation: list of MLPModelGenerator objects
    max_generations: int
    current_generation_number: int
    desired_fitness_score: float
    
    Returns:
    new_genepool: list of MLPModelGenerator objects
    """
       
    while current_generation_number < max_generations:
        current_generation_number += 1
        
        current_generation = sorted(current_generation, key=lambda x: x.fitness_score, reverse=True)
        print(f"Generation {current_generation_number} best model: {current_generation[0].genetic_pool} with fitness score: {current_generation[0].fitness_score}")
        print(f"Generation {current_generation_number} worst model: {current_generation[-1].genetic_pool} with fitness score: {current_generation[-1].fitness_score}")
        print(f"Generation {current_generation_number} average fitness score: {np.mean([model.fitness_score for model in current_generation])}")
        print(f"Generation {current_generation_number} median fitness score: {np.median([model.fitness_score for model in current_generation])}")
        print(f"Generation {current_generation_number} standard deviation of fitness score: {np.std([model.fitness_score for model in current_generation])}")
        print(f"Generation {current_generation_number} variance of fitness score: {np.var([model.fitness_score for model in current_generation])}")
        print(f"Generation {current_generation_number} range of fitness score: {np.ptp([model.fitness_score for model in current_generation])}")
        
        print("-------------------------------------------------")
        
        new_genepool = []
        if current_generation[0].fitness_score >= desired_fitness_score:
            return current_generation
        for i in range(0, len(current_generation), 2):
            print(f"Creating children from parents {i} and {i+1} in generation {current_generation_number} in loop {i/2+1}")
            parent1 = current_generation[i]
            if i+1 == len(current_generation):
                parent2 = current_generation[0]
            else:
                parent2 = current_generation[i+1]
            child1 = MLPModelGenerator(genetic_pool={key: parent1.genetic_pool[key] if nprandom.randint(0, 2) == 0 else parent2.genetic_pool[key] for key in parent1.genetic_pool.keys()})
            child2 = MLPModelGenerator(genetic_pool={key: parent1.genetic_pool[key] if nprandom.randint(0, 2) == 0 else parent2.genetic_pool[key] for key in parent1.genetic_pool.keys()})
            new_genepool.extend([child1, child2])
            
        # sort the genepool and make new genepool the same size as the initial genepool
        new_genepool = sorted(new_genepool, key=lambda x: x.fitness_score, reverse=True)[:initial_genepool_size]
        
        for i in range(0, len(new_genepool)):
            if nprandom.uniform() < 0.1:
                new_genepool[i] = MLPModelGenerator(fitness=True)
                
        # using recursion to call the genetic_algorithm function until the desired fitness score is reached
        new_genepool = genetic_algorithm(new_genepool, max_generations, current_generation_number, desired_fitness_score)
                
    return new_genepool

In [10]:
final_genepool = genetic_algorithm(initial_genepool, max_generations, current_generation, desired_fitness_score)

print(f"Final genepool: {final_genepool}")
print(f"Final best model: {final_genepool[0].genetic_pool} with fitness score: {final_genepool[0].fitness_score}")
print(f"Final worst model: {final_genepool[-1].genetic_pool} with fitness score: {final_genepool[-1].fitness_score}")
print(f"Final average fitness score: {np.mean([model.fitness_score for model in final_genepool])}")
print(f"Final median fitness score: {np.median([model.fitness_score for model in final_genepool])}")
print(f"Final standard deviation of fitness score: {np.std([model.fitness_score for model in final_genepool])}")
print(f"Final variance of fitness score: {np.var([model.fitness_score for model in final_genepool])}")
print(f"Final range of fitness score: {np.ptp([model.fitness_score for model in final_genepool])}")


Generation 1 best model: {'hidden_layer_sizes': [26, 11, 17], 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.005861122115926577, 'batch_size': 722, 'learning_rate': 'constant', 'learning_rate_init': 0.05201255233270858, 'power_t': 0.6271800903325878, 'max_iter': 238, 'shuffle': False, 'random_state': 76, 'momentum': 0.19576977721418665, 'nesterovs_momentum': True, 'beta_1': 0.3544286298666228, 'beta_2': 0.6456518606474096, 'epsilon': 9.915783762248037e-08, 'n_iter_no_change': 15} with fitness score: 0
Generation 1 worst model: {'hidden_layer_sizes': [21, 22, 16], 'activation': 'relu', 'solver': 'lbfgs', 'alpha': 0.011781474101609101, 'batch_size': 292, 'learning_rate': 'invscaling', 'learning_rate_init': 0.03468833974988981, 'power_t': 0.45428079889170025, 'max_iter': 498, 'shuffle': False, 'random_state': 8, 'momentum': 0.3917029497978216, 'nesterovs_momentum': True, 'beta_1': 0.3918443293447217, 'beta_2': 0.9718403454027018, 'epsilon': 7.978929644709549e-08, 'n_iter_no_change': 11